# Format the data 

In [ ]:
# imports
import pandas as pd
import json
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
import random


In [ ]:
# load the datasets
def load_dataset(file_path):
    with open(file_path, 'r') as f:
        data = pd.read_csv(file_path, encoding='utf-8-sig')
    return pd.DataFrame(data)

In [ ]:
# load the 3 dataset and process them accordingly

viet_data =load_dataset("../data/vietnamese_train_samples.csv")
fingpt_data = load_dataset("../data/fingpt_train_samples.csv")
synthesized_data = load_dataset("../data/synthetic_data_generate.csv")



In [ ]:
# load and proces the validation dataset
viet_data_val = load_dataset("../data/vietnamese_validation_samples.csv")
fingpt_data_val = load_dataset("../data/fingpt_validation_samples.csv")

In [ ]:
print(viet_data_val.head())


                                            sentence sentiment     topic  \
0                                     em thấy ổn ạ .   neutral    others   
1     giảng viên lên lớp đúng giờ , dạy nhiệt tình .  positive  lecturer   
2                                  tại do tuần thi .   neutral    others   
3  thầy dạy hài hước , tận tâm trước các vấn đề v...  positive  lecturer   
4              giảng viên dạy nhiệt tình , tận tâm .  positive  lecturer   

   length  word_count                                          tokenized  \
0      14           5                                     em thấy ổn ạ .   
1      46          11     giảng_viên lên_lớp đúng giờ , dạy nhiệt_tình .   
2      17           5                                  tại do tuần thi .   
3      89          22  thầy dạy hài_hước , tận_tâm trước các vấn_đề v...   
4      37           9              giảng_viên dạy nhiệt_tình , tận_tâm .   

                                    sentence_cleaned  \
0                             

2. Data Transformation

In [ ]:
# transform the data to instruction format

for i in range(len(fingpt_data)):
    input = fingpt_data['input'][i]
    instruction = fingpt_data['instruction'][i]
    user_msg = "\"" + input + "\"" + " " + instruction
    fingpt_data.at[i, 'user_msg'] = user_msg

vi_instruction = "Phân loại cảm xúc của câu này là gì? Hãy chọn câu trả lời từ {negative/neutral/positive}."
for i in range(len(viet_data)):
    input = viet_data['sentence'][i]
    topic = viet_data['topic'][i]
    user_msg = "\"" + input + "\"" + " " + vi_instruction + " Đây là chủ đề: " + topic
    viet_data.at[i, 'user_msg'] = user_msg
    
for i in range(len(synthesized_data)):
    input = synthesized_data['input'][i]
    instruction = synthesized_data['instruction'][i]
    user_msg = "\"" + input + "\"" + " " + instruction
    synthesized_data.at[i, 'user_msg'] = user_msg


In [ ]:
for i in range(len(viet_data_val)):
    input = viet_data_val['sentence'][i]
    topic = viet_data_val['topic'][i]
    user_msg = "\"" + input + "\"" + " " + vi_instruction + " Đây là chủ đề: " + topic
    viet_data_val.at[i, 'user_msg'] = user_msg
for i in range(len(fingpt_data_val)):
    input = fingpt_data_val['input'][i]
    instruction = fingpt_data_val['instruction'][i]
    user_msg = "\"" + input + "\"" + " " + instruction
    fingpt_data_val.at[i, 'user_msg'] = user_msg

In [ ]:
fingpt_subset = pd.DataFrame({
    'user_msg': fingpt_data['user_msg'],
    'output': fingpt_data['output'],
})

viet_subset = pd.DataFrame({
    'user_msg': viet_data['user_msg'],
    'output': viet_data['sentiment'],
})

synth_subset = pd.DataFrame({
    'user_msg': synthesized_data['user_msg'],
    'output': synthesized_data['output'],
})

combined_data = pd.concat([fingpt_subset, viet_subset, synth_subset], ignore_index=True)
combined_data = combined_data.sample(frac=1, random_state=1).reset_index(drop=True)
combined_data.head()

combined_data.to_csv("../data/combined_data.csv", index=False, encoding='utf-8-sig')

In [ ]:
viet_val_subset = pd.DataFrame({
    'user_msg': viet_data_val['user_msg'],
    'output': viet_data_val['sentiment'],
})
fingpt_val_subset = pd.DataFrame({
    'user_msg': fingpt_data_val['user_msg'],
    'output': fingpt_data_val['output'],
})
combined_val_data = pd.concat([viet_val_subset, fingpt_val_subset], ignore_index=True)
combined_val_data = combined_val_data.sample(frac=1, random_state=1).reset_index(drop=True)
combined_val_data.head()


,user_msg,output
0,"""Is Now The Time To Look At Buying Avaya Holdi...",neutral
1,"""'Mad Money' host Jim Cramer and the 'Squawk o...",negative
2,"""mong thầy vẫn giữa phong độ như thế ;) ."" Phâ...",neutral
3,"""nhiệt tình , tận tâm , hài hước , famous name...",positive
4,"""The Department of Justice could sue Apple by ...",neutral


In [ ]:
combined_val_data.to_csv("../data/combined_val_data.csv", index=False, encoding='utf-8-sig')

3. Template Application
    - Apply the appropriate chat template for your model
    - Insert all necessary special tokens
    - Format conversations correctly for the tokenizer